In [1]:
from pathlib import Path
import polars as pl
import polars.selectors as cs
import pandas as pd

In [2]:

OUTDIR = "/project/def-nahee/kbas/POM_Response_Parquet"
TRAIN_FEATURE_DIR = Path(OUTDIR) / "merged" / "train" / "features"

In [3]:
TRAIN_FEATURE_DIR

PosixPath('/project/def-nahee/kbas/POM_Response_Parquet/merged/train/features')

In [4]:
pattern = str(TRAIN_FEATURE_DIR / "features_*.parquet")
lf = pl.scan_parquet(pattern)

schema = lf.collect_schema()
num_cols = [c for c, t in schema.items() if t.is_numeric()]
exclude = {"event_no", "global_event_no"}
num_cols = [c for c in num_cols if c not in exclude]

q25_df = lf.select([pl.col(c).quantile(0.25).alias(c) for c in num_cols]).collect(streaming=True)
q50_df = lf.select([pl.col(c).quantile(0.50).alias(c) for c in num_cols]).collect(streaming=True)
q75_df = lf.select([pl.col(c).quantile(0.75).alias(c) for c in num_cols]).collect(streaming=True)

quantiles = pd.DataFrame({
    "feature": num_cols,
    "p25": [q25_df[c][0] for c in num_cols],
    "p50": [q50_df[c][0] for c in num_cols],
    "p75": [q75_df[c][0] for c in num_cols],
})
quantiles


/tmp/ipykernel_3030621/396948655.py:9: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  q25_df = lf.select([pl.col(c).quantile(0.25).alias(c) for c in num_cols]).collect(streaming=True)
/tmp/ipykernel_3030621/396948655.py:10: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  q50_df = lf.select([pl.col(c).quantile(0.50).alias(c) for c in num_cols]).collect(streaming=True)
/tmp/ipykernel_3030621/396948655.py:11: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  q75_df = lf.select([pl.col(c).quantile(0.75).alias(c) for c in num_cols]).collect(streaming=True)


,feature,p25,p50,p75
0,charge,0.827317,1.044017e+00,1.289065
1,dom_time,2397.685501,3.187647e+03,4310.009759
2,dom_x,-320.000000,0.000000e+00,320.000000
3,dom_y,-315.025774,-3.789764e+01,308.512517
4,dom_z,-200.000000,5.000000e+01,300.000000
5,string,85.000000,1.700000e+02,256.000000
6,pmt_number,4.000000,8.000000e+00,12.000000
7,dom_number,6.000000,1.100000e+01,16.000000
8,pmt_x,-320.000000,-2.130940e-17,320.000000
9,pmt_y,-315.117017,-3.771556e+01,308.603760


In [5]:
out_dir = Path("/project/def-nahee/kbas/Graphnet-Applications/MyClasses")
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = out_dir / "train_feature_percentiles_p25_p50_p75.csv"
quantiles.to_csv(csv_path, index=False, float_format="%.16f")  

print("saved:", csv_path)

saved: /project/def-nahee/kbas/Graphnet-Applications/MyClasses/train_feature_percentiles_p25_p50_p75.csv
